In [1]:
import torch
from torchvision.datasets import STL10
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Data transformation (resize and convert to grayscale)
transform = transforms.Compose([
    transforms.Grayscale(),                # Convert the image to grayscale
    transforms.Resize((32, 32)),           # Resize the image to 32x32
    transforms.ToTensor(),                 # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))   # Normalize the image
])

In [3]:
# Load training and test datasets
train_dataset = STL10(root='./data', split='train', download=True, transform=transform)
test_dataset = STL10(root='./data', split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Load data into DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=5000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [5]:
# Get the training and test data
train_data_iter = iter(train_loader)
test_data_iter = iter(test_loader)

train_images, train_labels = next(train_data_iter)
test_images, test_labels = next(test_data_iter)

In [6]:
# Convert to 2D array and display
train_images = train_images.view(train_images.size(0), -1).numpy()  
test_images = test_images.view(test_images.size(0), -1).numpy()

print(f"Train Data Shape: {train_images.shape}")
print(f"Test Data Shape: {test_images.shape}")

Train Data Shape: (5000, 1024)
Test Data Shape: (1000, 1024)


In [7]:
# Train the Logistic Regression model
# Set parameters|
log_reg = LogisticRegression(max_iter=1000,solver='lbfgs') 

log_reg.fit(train_images, train_labels)  # Fit the model on training data

# Make predictions on the test data
test_predictions = log_reg.predict(test_images)

In [8]:
# Calculate the accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 24.00%


In [9]:
import matplotlib.pyplot as plt

# Define the classes in STL-10
classes = ['Airplane', 'Bird', 'Car', 'Cat', 'Deer', 'Dog', 'Horse', 'Monkey', 'Ship', 'Truck']

In [12]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [16]:
#Prediction probabilities
test_probabilities = log_reg.predict_proba(test_images)

In [17]:
# ROC Curve
from sklearn.preprocessing import label_binarize

# Binarize the labels for multi-class ROC
test_labels_binarized = label_binarize(test_labels, classes=np.arange(len(classes)))

# Plot ROC curve for each class
plt.figure(figsize=(10, 7))
for i in range(len(classes)):
    fpr, tpr, _ = roc_curve(test_labels_binarized[:, i], test_probabilities[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{classes[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()



In [18]:
# Display first 5 test images 
for i in range(5):
    img = test_images[i].reshape(32, 32)  # Reshape the image to 32x32 
    true_label = test_labels[i]
    predicted_label = test_predictions[i]

    plt.subplot(1, 5, i + 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"T:{classes[true_label]}\nP:{classes[predicted_label]}")
    plt.axis('off')
plt.show()